# Avaliação Contínua V
## Introdução à Inteligência Artificial, edição 23/24

<img src="Imagens\220px-Sokoban_ani.gif" alt="Drawing" style="width: 200px;"/>

## Introdução

Nesta avaliação contínua irão modelizar o problema do Sokoban usando o python através de uma variação do código disponibilizado pelo aima-python tendo para isso também que contornar alguma ineficiência na transformação dos esquemas de acções nas suas diversas instâncias, a qual limita a modelização dos problemas de planeamento com esquemas de acções que envolvam muitos objectos e várias variáveis. 


### Recursos necessários
Para este projecto os seguintes módulos são necessários (distribuídos juntamente com o enunciado):
* `planningPlus.py` - módulo principal
* `search.py` - módulo principal
* `logic.py` - módulo auxiliar
* `utils.py` - módulo auxiliar

## O problema do Sokoban
<img src="Imagens\4c8750980f04a2fb9dbea2230db9728904f37297-sokoban-start-to-end.PNG" alt="Drawing" style="width: 400px;"/> 

O puzzle Sokoban é jogado tipicamente numa grelha de células quadradas, onde cada célula é uma parede ou chão navegável. Algumas das células de chão contêm caixas e outras estão marcadas como lugares de armazenamento das caixas.

O jogador (Sokoban) está confinado ao tabuleiro e pode movimentar-se ortogonalmente para células adjacentes, que sejam navegáveis e vazias - o sokoban não tem super poderes, não podendo atravessar nem paredes nem caixas.

O Sokoban pode também empurrar caixas que estejam ao seu lado e pode fazê-lo para células vazias adjacentes. As caixas não podem ser empurradas se ao seu lado, na orientação do movimento, estiver uma parede ou outra caixa. 

O número de caixas é sempre igual ao número de destinos de armazenamento. O puzzle fica resolvido quando todos os lugares de armazenamento forem ocupados por caixas.

Pretendemos que formulem um qualquer puzzle Sokoban, dado em modo txt, como um problema de planeamento usando PDDL (*Planning Domain Definition Language*). 

## A linguagem PDDL
Em princípio, para definirmos um problema temos de ter:

* um conhecimento base
* um conjunto de objetivos
* um conjunto de esquemas de ações
* O domínio

Vejam que:
* o conhecimento de base é constituído pelos objectos e conjunção de expressões lógicas (positivas e negativas) que representam a situação inicial que enfrenta o planeador. O conhecimento de base corresponde ao estado no paradigma do espaço de estados, onde se representa a informação que muda com as acções.
* Os objectivos serão uma lista de expressões lógicas positivas e negaticas que terão ser verdadeiras para que sejam cumpridos.
* O conjunto de esquemas de acções possuem pré-condições, efeitos e domínios. Na prática são fábricas que permitem gerar todas as acções possíveis para todas as permutações das suas variáveis, considerando que têm de satisfazer os respectivos domínios.
* No domínio representamos a informação estática que não muda com as acções. 

## Sokoban em modo texto
Vamos representar os puzzles Sokoban em .txt usando os símbolos seguintes:
```python
#   (cardinal)   Parede
o   (ó)          Objectivo vazio
@   (arroba)     Sokoban no chão
+   (soma)       Sokoban num objectivo
$   (dólar)      Caixa no chão
*   (asterisco)  Caixa no objectivo
```
Por exemplo o puzzle na imagem
<img src="imagens\grandeSokoba.PNG" alt="Drawing" style="width: 175px;"/> 
corresponderá ao puzzle .txt:
```python

                                               #####
                                             ###...#
                                             #o@$..# 
                                             ###.$o#
                                             #o##$.#
                                             #.#.o.##
                                             #$.*$$o#
                                             #...o..#
                                             ########
```

### 1. Modelização do Sokoban
Um dos objectivos é a modelização de qualquer problema de Sokoban como um problema de planeamento usando a linguagem PDDL definida no aima-python. É preciso modelizar as expressões lógicas para exprimir o conhecimento envolvido em qualquer puzzle de Sokoban e os esquemas de acções que permitem ao planeador encontrar o plano para colocar as caixas nos objectivos. Para cada puzzle concreto é preciso representar o conhecimento de base que vai corresponder ao estado inicial do planeador. E é necessário também modelizar os domínios.  

Essa modelização é livre, não impomos nada. Vamos fornecer um puzzle Sokoban em modo texto que terão de converter numa instância de `PlanningProblem`, criando o conhecimento de base e os objectivos. No entanto, para resolver um problema de ineficência do código do aima-python como irão ver, não irão ser necessário nem os domínios nem os esquemas das acções. 
Devido à elevada ineficência do código do aima-python, concretamente o método `expand_actions` da classe `PlanningProblem`, voçês terão de criar código python que gere automaticamente as acções para um dado puzzle.

### 2. As ineficiências da geração das ações a partir dos esquemas de acções
O método `expand_actions` é responsável pela geração das acções concretas a partir dos esquemas das acções.
Vejemos como funciona através de um exemplo:
<img src="imagens\hanoi.PNG" alt="Drawing" style="width: 175px;"/> 

Se tivermos a seguinte modelização do Problema das Torres de Hanói,


In [ ]:
from planningPlus import *
from logic import *
from utils import *
from search import *

In [ ]:
estado = [
    expr('Sobre(D1,D2)'),
    expr('Sobre(D2,D3)'),
    expr('Sobre(D3,A)'),
    expr('Livre(D1)'),
    expr('Livre(B)'),
    expr('Livre(C)')
]

dominio = 'Disco(D1) & Disco(D2) & Disco(D3) & Pino(A) & Pino(B) & Pino(C) & Menor(D1,D2) & ' + \
            'Menor(D2,D3) & Menor(D1,D3) & Menor(D1,A) & Menor(D1,B) & Menor(D1,C) & ' + \
            'Menor(D2,A) & Menor(D2,B) & Menor(D2,C) & Menor(D3,A) & Menor(D3,B) & Menor(D3,C)'

goal = 'Sobre(D3,C) & Sobre(D2,D3) & Sobre(D1,D2)'

acoes = [Action('Move(d,x,y)', 
               precond='Livre(d) & Sobre(d,x) & Livre(y)',
               effect='Sobre(d,y) & Livre(x) & ~Sobre(d,x) & ~Livre(y)',
               domain='Disco(d) & Menor(d,x) & Menor(d,y)')]

th = PlanningProblem(initial=estado, goals=goal, actions=acoes, domain=dominio)
p = ForwardPlan(th)

Ao expandirmos as acções teremos $38$ acções instanciadas com os 6 objectos do domínio, os 3 discos e os 3 pinos:

In [ ]:
th.expand_actions()

Na prática, o que o método `expand_actions` faz é gerar todas as permutações dos tuplos $(b,x,y)$ em que qualquer dos seus elementos pode ser um dos objectos. Como não há repetições, teremos $6 \times 5 \times 4$ permutações, i.e. $120$. Dessas permutações, que serão tentadas todas, filtram-se as que não satisfazem os domínios, neste caso, $b$ tem que ser um disco e $b$ tem de ser menor do que $x$ e do que $y$.
Por exemplo, umas das permutações é $(A,B,C)$, que representa mover o pino $A$ do pino $B$ para o pino $C$, e que falha na satisfação dos domínios: $A$ não é disco. Mais ou menos dois terços das permutações irão para o lixo, mas gasta-se tempo no processo de geração de todas as permutações e na verificação dos domínios.

Se tivermos 30 objectos, imaginem que temos $27$ discos e os mesmos $3$ pinos, isso quer dizer que serão geradas $30 \times 29 \times 28=24360$ permutações e serão filtradas todas as que não satisfaçam os domínios. Se tivermos os mesmos $30$ objectos e tivermos esquemas de acções que envolvem $4$ parâmetros, teremos que verificar $30 \times 29 \times 38 \times 27$ permutações, neste caso: $657720$. Com esta clara inefiência do `expand_actions` o processo de planeamento não escala para problemas mais complexos, o que é o caso dos puzzles mais simples do Sokoban.

Reparem que sempre que construímos uma instância da classe `ForwardPlan`, que é uma subclasse de `Problem`, a qual recebe uma instância de `PlanningProblem`, as ações são expandidas no construtor, através da invocação do método `expand_actions`, ficando guardadas no atributo `expanded_actions`. Isso quer dizer que para um considerável número de objectos envolvidos, a instânciação dos esquemas que envolvem $3$ ou mais variáveis é insuportavelmente lento, pondo em risco a solução dos puzzles do Sokoban com grelhas de uma dimensão razoável, envolvendo várias caixas.

## Objectivos
<img src="Imagens\sokobanYY.gif" alt="Drawing" style="width: 180px;"/>

Pretendemos que desenvolvam em python a função `sokoban` que recebe como input uma representação em modo txt de um qualquer puzzle Sokoban e que devolva a instância de `ForwardPlan` com o estado inicial, os objectivos e as acções expandidas, de modo a podermos resolver esse puzzle através de um método de procura standard.

Notem que para isso, podem gerar uma instância de `PlanningProblem` com o conhecimento de base e os objectivos, não terão de criar os outros dois argumentos: o esquema de acções e os domínios, que podem ser ambos listas vazias. Essa instância de `PlanningProblem` será depois usada como input na construção da instância de `ForwardPlan`. O método `expand_actions` será aplicado a uma lista vazia de esquemas de acções gerando também uma lista vazia. Assim, depois de construirem a instância de `ForwardPlan` terão de gerar todos as acções para esse problema, que guardam no atributo `expanded_actions`. Toda a informação que é preciso para gerar as acções é passada no puzzle txt de input e elas terão de satisfazer o vosso modelo de acções. A intenção é curto-circuitar o método `expand_actions`, construindo logo as acções válidas. Essa instância de `ForwardPlan` estará então pronta a ser usada por um método de procura para encontrar uma solução. 

Para diminuirem a dimensão do espaço de estados, obrigando o algoritmo de procura a "antecipar um falhanço" e evitando assim explorar uma subárvore de estados sem sucesso, devem também considerar não gerar acções que levem as caixas para cantos que não sejam destinos de caixas. Desse modo, diminuiem o número de acções expandidas e aumentam a eficácia do processo de procura. 

**Melhoria**: Uma sugestão para diminuirem ainda mais o espaço de estado pode ser não criarem acções que empurrem as caixas para linhas ou colunas de fronteira onde não haja destinos para as caixas. Na imagem do puzzle em cima as linhas de topo e de fundo, formadas ambas por $3$ células, são exemplos de linhas fronteira e o mesmo acontece com a coluna mais à direita formada por $6$ células. Uma caixa numa coluna fronteira só pode ser deslocada verticalmente ou horizontalmente, ficando bloqueada nessa linha ou coluna. Se não existir nenhum célula objectivo nesse segmento de linha ou coluna com caixa, quaisquer acções a partir daí levam inevitavelmente a becos sem saída, atrasando o processo de procura. Será assim ideal evitar levar caixas para essas armadilhas.

Para os nossos testes, como não conhecemos as acções que modelizaram, iremos usar uma procura em largura em grafo, a qual obtém a solução óptima, e contamos o número de acções da solução. Assim, não dependeremos de qualquer ordem de acções.

## Exemplos

### Exemplo 1
```python
##########
#        #
#  $  +  #
#        #
##########
Sol:W-W-N-W-W-S-E-E-E   - 9 passos
```



### Exemplo 2
```python
####
# o#
#  ###
#*@  #
#  $ #
#    #
######
Sol:S-S-E-E-S-W-N-N-N  - 9 passos
```

### Exemplo 3
```python
#######
#     #
#  $  #
#     #
#@    #
#     #
#o    #
#######
Sol: E-E-E-N-N-W-W-N-W-S-S-S-S - 13 passos
```

### Exemplo 4
```python
#######
# o   #
# # # #
# # # #
# $@  #
#     #
#######
Sol:S-W-W-N-E-S-E-N-N-N-S-S-E-E-E-N-N-N-W-W-W  - 21 passos
```


### Exemplo 5
```python
  #####
###...#
#.@$..#
###..o#
#o##..# 
#.#...##
#$.....#
#......#
########
Sol:E-N-E-S-W-S-E-S-S-W-S-W-S-W-N-N em 16 passos 
```

Exemplo do teste5 em python

```python
linha1= "  ##### \n"
linha2= "###...# \n"
linha3= "#o@$..# \n"
linha4= "###.$o# \n"
linha5= "#o##..# \n"
linha6= "#.#...##\n"
linha7= "#$.....#\n"
linha8= "#......#\n"
linha9= "########\n"
grelha=linha1+linha2+linha3+linha4+linha5+linha6+linha7+linha8+linha9
p=sokoban(grelha)
travel_sol = breadth_first_graph_search_plus(p)
if travel_sol:
    print('Solução em :',len(travel_sol.solution()),'acções')
else:
    print('No Way!')
``` 

## Submissão

### Quizz
Cada grupo deve completar a implementação da classe pedida e testá-la no link do *quizz* **Avaliação Contínua v** introduzindo e testando o vosso código. 

Como dissemos atrás, este *quizz* é constituído por 3 perguntas. As 3 perguntas serão avaliadas através de um conjunto de testes automáticos visíveis e mais alguns testes escondidos, valendo no total 1.75 valores.

Podem ir verificando o código (botão check) e submeterem as vezes que quiserem (ambos os elementos do grupo), sendo a submissão com melhor nota a que será considerada.

### Prazo
A submissão fecha às 23:59 de Terça, 12 de Dezembro.

### Ficheiro Python
Simultaneamente é necessario submeter o ficheiro Python, que contém todo o código das 3 perguntas, na página da disciplina. Só queremos uma submissão por grupo. Esse ficheiro deve chamar-se *sokoban_IIA_23_24_grupoXX.py* em que substituem XX pelo identificador do grupo. 

<img src="Imagens\sokobanXX.gif" alt="Drawing" style="width: 200px;"/>